In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

import dill

pd.options.display.max_colwidth = None

In [4]:
# recipes
recipes = pd.read_csv('Data/recipes_50k.csv')
# recipes.drop(columns = 'Unnamed: 0', inplace = True)
# nutrition = pd.read_csv('Data/nutrition_clean.csv')
# nutrition.drop(columns = 'Unnamed: 0', inplace = True)
recipes = recipes[:20_000]
# recipes = pd.concat([recipes, nutrition], axis = 1)

# recipes.dropna(inplace = True)
# # remove the obscenely long recipes
# recipes = recipes[recipes['minutes'] <= 540]

# # correct dataframe so that everything will work
# recipes.reset_index(drop = True, inplace = True)



# recipes = recipes[:50_000]
# # nutrition = pd.read_csv('Data/nutrition_clean.csv')
# # nutrition.drop(columns = 'Unnamed: 0', inplace = True)

In [5]:
recipes.to_csv('Data/recipes_50k.csv')

In [34]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     30000 non-null  int64  
 1   name           30000 non-null  object 
 2   id             30000 non-null  int64  
 3   minutes        30000 non-null  int64  
 4   tags           30000 non-null  object 
 5   n_steps        30000 non-null  int64  
 6   steps          30000 non-null  object 
 7   description    30000 non-null  object 
 8   ingredients    30000 non-null  object 
 9   n_ingredients  30000 non-null  int64  
 10  calories       30000 non-null  float64
 11  tot_fat        30000 non-null  float64
 12  sugar          30000 non-null  float64
 13  sodium         30000 non-null  float64
 14  protein        30000 non-null  float64
 15  sat_fat        30000 non-null  float64
dtypes: float64(6), int64(5), object(5)
memory usage: 3.7+ MB


In [15]:
# recipes.to_csv('Data/recipes_50k.csv')

In [16]:
# create series witht the recipe names as the index
new_desc = pd.Series(recipes.index[:20_001], index = recipes['name'][:20_001])

In [17]:
# pull out ingredients to vectorize
ingredients = recipes[['ingredients', 'name', 'minutes']].copy()
descriptions = recipes[['description', 'name']].copy()


# drop null values
# recipes.dropna(inplace = True)

In [18]:
def list_similarity(new_list = None, test = None):
    # set count
    count = 0
    
    test_list = test.split()
    
    # check if ingredient is in the string and if it is, add to the count
    for x in new_list:
        if x in test:
            count+= 1
            
    # percent of ingredients that are identical between the two recipes
    return count / len(test), count

In [19]:
def in_pantry(on_hand = None, min_score = 0, min_ing = 0):
    # empty list for output
    recipe_list = []
    
    # recipe database
    ing_list = [ingredients.iloc[x] for x in range(20_001)]
    
    # iterate through list
    for x in range(len(ing_list)):
        
        # get score and count
        sim_score, count = list_similarity(on_hand, ing_list[x][0])
        if  sim_score >= min_score and count >= min_ing:
            recipe_list.append(ing_list[x][1])
    
    index_list = [new_desc[x] for x in recipe_list]
    # output
    x = pd.DataFrame(recipes.iloc[new_desc[recipe_list[0]]]).T
    
    # create a DataFrame
    for num in index_list:
        y = pd.DataFrame(recipes.iloc[new_desc[num]]).T
        x = x.append(y)
    
    return x[['name','description','ingredients', 'steps', 'minutes', 'calories', 'tot_fat', 'protein']].sample().T

In [30]:
in_pantry(on_hand = ['pasta', 'vegetable stock', 'chicken', 'quinoa'], min_ing = 2)

,12136
name,bacon soup with veggies
description,"dinner is ready for 15 mins,so easy...perfect on this cold weather"
ingredients,"['cooked bacon', 'garlic', 'onion', 'italian seasoning', 'cut green beans', 'star-shaped pasta', 'dried parsley', 'kosher salt', 'olive oil', 'carrot', 'potatoes', 'chicken broth']"
steps,"['1', 'chop or scramble the bacon into small pieces , set aside', '2', 'heat the olive oil in skillet , add garlic and wait until its golden brown', '3', 'add the chopped onion and bacon , stir', '4', 'season with italian seasoning , add potatoes , parsley , and broth', 'lets boil then add the small pasta or pastina', '5', 'simmer for 10 mins then add the carrot and beans , season with salt and serve with garlic bread / toasted bread or baguet , italian bread']"
minutes,16
calories,221.9
tot_fat,15
protein,14
